# Recipe Assistant MVP
**Project:** Personalized and Conscious Recipe Assistant Agent

*Initial code (MVP) for a conversational recipe assistant using Google Gemini SDK in Google Colab. This MVP focuses on using Gemini to generate recipes and adaptations directly based on user input, simulating search/creation.*





In [ ]:
import google.generativeai as genai
import os
from IPython.display import display, Markdown
from google.colab import userdata


global_chat_persona = "Personal Recipe Assistant"

# Configure the Google Gemini API
try:
    google_api_key = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = google_api_key
    if not google_api_key:
        raise ValueError("API Key not found in Colab secrets.")
    genai.configure(api_key=google_api_key)
except ImportError:
    if not google_api_key:
        print("API Key not found in the environment variables.")
        google_api_key = None
except Exception as e:
    print(f"Error! API Key not found: {e}")
    google_api_key = None


## Agent prompts

In [ ]:
# 1. Understand the user's request
request_prompt = """
You are the main Orchestrator Agent for a Personalized and Conscious Recipe Assistant.
Your role is to understand the user's request, identify key information (ingredients, restrictions, desired meal type, number of servings, specific adaptations like Airfryer), and coordinate the work of specialized recipe agents.

Based on the user's input, extract the following information:
- Ingredients provided (list)
- Dietary restrictions (list, e.g., "gluten-free", "vegan", "no nuts")
- Meal type (e.g., "lunch", "dinner", "snack")
- Number of servings (integer, default to 2 if not specified)
- Adaptation requests (list, e.g., "Airfryer")

Present the extracted information in a clear, structured format (e.g., JSON-like structure or key-value pairs) so the main program can easily parse it and call the appropriate specialized agents. Do NOT generate the recipe yourself. Simply extract and format the request details.

Example Output Format:
{
  "ingredients": ["chicken", "broccoli", "rice"],
  "restrictions": ["gluten-free"],
  "meal_type": "lunch",
  "servings": 2,
  "adaptations": ["Airfryer"]
}

If the request is unclear or missing crucial information (like ingredients), ask the user for clarification.
"""

# 2. Recipe generation based on ingredients and meal type.
recipe_generator_prompt = """
You are a Recipe Generation Agent. Your task is to create a base recipe based on a list of ingredients and a meal type provided by the Orchestrator Agent.

Generate a complete recipe including:
- Title
- Brief description
- List of ingredients with suggested quantities
- Step-by-step instructions for traditional preparation (stove, oven, etc.)

The recipe should primarily use the ingredients provided. If necessary, you may suggest common pantry staples (salt, pepper, oil) but focus on the core ingredients given.
Present the recipe in a clear, structured format using Markdown.
"""

# 3. Restriction adaptation to modify a recipe to meet specific dietary restrictions.
restriction_adapter_prompt = """
You are a Restriction Adaptation Agent. Your task is to take an existing recipe and modify it to comply with a list of dietary restrictions provided by the Orchestrator Agent.

Analyze the provided recipe and the restrictions. Suggest substitutions or modifications to ingredients and/or instructions to make the recipe compliant.

Present the modified recipe or a clear list of changes needed, maintaining the original structure as much as possible. Use Markdown for clarity.
If the recipe already meets the restrictions, state that. If necessary, use Google (google_search) to adapt the recipe.
"""

# 4. Airfryer adaptation
airfryer_adapter_prompt = """
You are an Airfryer Adaptation Agent. Your task is to take an existing recipe and provide instructions on how to cook it using an Airfryer. If necessary, use Google (google_search) to adapt the recipe.

Analyze the provided recipe. Determine appropriate temperatures, cooking times, and specific steps needed to prepare this dish in an Airfryer.

Provide a separate section detailing the "Airfryer Adaptation". Include suggested temperature, estimated time, and any specific instructions (like preheating, shaking the basket, cooking in batches, etc.). Use Markdown for clarity.
"""

## Agents

In [ ]:
# google agent framework
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner # orquestrador
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
import json
import time


class AgentResponseObject:
    def __init__(self, text_content: str):
        self.text = text_content

default_model = 'gemini-2.5-flash-preview-04-17'

# --- agent orchestrator function ---
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response_text = ""

    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response_text += part.text
    return AgentResponseObject(final_response_text)

# --- agents ---
def request_agent(input):
  agent = Agent(
      name='request_agent',
      model=default_model,
      instruction=request_prompt,
      description="Understand the user's request"
  )
  return call_agent(agent, input)


def recipe_generator_agent(input):
  agent = Agent(
      name='recipe_generator',
      model=default_model,
      instruction=recipe_generator_prompt,
      description="Recipe generation based on ingredients"
  )
  return call_agent(agent, input)


def restriction_adapter_agent(input):
  agent = Agent(
      name='restriction_adapter',
      model=default_model,
      instruction=restriction_adapter_prompt,
      description="Adaptation to modify a recipe to meet specific dietary restrictions",
      tools=[google_search]
  )
  return call_agent(agent, input)


def airfryer_adapter_agent(recipe):
  agent = Agent(
      name='airfryer_adapter',
      model=default_model,
      instruction=airfryer_adapter_prompt,
      description="Adaptat recipe to use airfryer",
      tools=[google_search]
  )
  return call_agent(agent, recipe)


## Main Orchestration

In [ ]:
def process_user_request(user_input):
    try:
        # Step 1: Parse the user's request
        print("\nProcessing your request...")
        request_response = request_agent(f"User request: {user_input}\n\nExtract details in JSON format.")
        request_details_str = request_response.text.strip()

        # parsing JSON-like string
        try:
            if request_details_str.startswith("```json"):
                request_details_str = request_details_str[len("```json"):].strip()
            if request_details_str.endswith("```"):
                request_details_str = request_details_str[:-len("```")].strip()
            request_details = json.loads(request_details_str)
            print(f"{global_chat_persona}: Your request details {request_details}")

        except json.JSONDecodeError as e:
            print(f"{global_chat_persona}: The request details coud not be parsed. Error: {e}")
            # If parsing fails, return the response directly, maybe the orchestrator asked for clarification
            return request_response.text

        ingredients = request_details.get("ingredients", [])
        restrictions = request_details.get("restrictions", [])
        meal_type = request_details.get("meal_type", "meal")
        servings = request_details.get("servings", 2) # Default to 2
        adaptations = request_details.get("adaptations", [])

        if not ingredients:
            return f"{global_chat_persona}: Please tell me what ingredients you have so I can suggest a recipe."

        # Step 2: Generate the base recipe
        print(f"{global_chat_persona}: Generating your recipe...")
        generator_input = f"Ingredients: {', '.join(ingredients)}\nMeal Type: {meal_type}\nServings: {servings}"
        base_recipe_response = recipe_generator_agent(generator_input)
        current_recipe_text = base_recipe_response.text
        # Optional: show base recipe
        # display(Markdown(current_recipe_text))

        # Step 3: Restriction Adaptation if requested
        if restrictions:
            print(f"{global_chat_persona}: Processing restriction adaptation as requested...")
            restriction_input = f"Original Recipe:\n{current_recipe_text}\n\nRestrictions: {', '.join(restrictions)}"
            restriction_response = restriction_adapter_agent(restriction_input)
            current_recipe_text = restriction_response.text

        # Step 4: Airfryer Adaptation if requested
        if any(word.lower() == "airfryer".lower() for word in adaptations):
            print(f"{global_chat_persona}: Processing Airfryer adaptation as requested...")
            airfryer_input = f"Original Recipe:\n{current_recipe_text}"
            airfryer_response = airfryer_adapter_agent(airfryer_input)
            current_recipe_text += "\n\n" + airfryer_response.text

        # Step 5: Return the final recipe
        return current_recipe_text

    except Exception as e:
        print(f"{global_chat_persona}: Error when processing your request: {e}")
        return "An error occurred while processing your request. Please try again."


In [86]:
# --- Main ---
# user interaction
if google_api_key:
    try:
        print("\nHey! I am your personal Recipe Assistant.\n\n")
        print("Tell me what ingredients you have, your restrictions, for how many people, and if you want to adapt it for an Airfryer.\n")
        print("Example: I have potatoes and cauliflower. I want a vegetarian and gluten-free dinner. Please adapt for the airfryer.")

        while True:
          user_input = input("\nRecipe request (or type 'exit' to quit): ")

          # exit condition
          if user_input.lower() == 'exit':
              print("\nEnjoy your meal... See you soon!\n\n ")
              break

          # Process using the multi-agent system
          final_response_text = process_user_request(user_input)

          print(f"\n{global_chat_persona}:")
          display(Markdown(final_response_text))

          if "An error occurrd while processing your request" in final_response_text:
              break
          time.sleep(0.1)
    except Exception as e:
        print(f"Error: {e}")

else:
    print(f"\n{global_chat_persona} could not be initialized. Check your API Key and try again.")



Hey! I am your personal Recipe Assistant.


Tell me what ingredients you have, your restrictions, for how many people, and if you want to adapt it for an Airfryer.

Example: I have potatoes and cauliflower. I want a vegetarian and gluten-free dinner. Please adapt for the airfryer.

Processing your request...
Personal Recipe Assistant: Your request details {'ingredients': ['carne', 'tomate', 'arroz'], 'restrictions': [], 'meal_type': 'jantar', 'servings': 2, 'adaptations': ['airfryer']}
Personal Recipe Assistant: Generating your recipe...
Personal Recipe Assistant: Processing Airfryer adaptation as requested...

Personal Recipe Assistant:


Okay, here is a simple recipe for a dinner dish using carne, tomate, and arroz for 2 servings.

# Carne Simples com Arroz e Tomate

Uma refeição reconfortante e fácil de preparar, perfeita para um jantar rápido usando ingredientes básicos. A carne é cozida num molho simples de tomate e servida sobre arroz soltinho.

**Para 2 porções**

---

**Ingredientes:**

*   250g de carne de vaca (cortada em cubos pequenos ou tiras finas)
*   1 lata (aprox. 400g) de tomate picado em lata
*   200g de arroz (aproximadamente 1 chávena)
*   400ml de água (para cozinhar o arroz)
*   1 cebola pequena (opcional, picada)
*   1 dente de alho (opcional, picado)
*   2-3 colheres de sopa de azeite ou óleo vegetal
*   Sal a gosto
*   Pimenta preta moída na hora a gosto

---

**Instruções:**

1.  **Preparar o Arroz:** Comece por cozinhar o arroz. Lave o arroz em água corrente se desejar. Numa panela, coloque o arroz e os 400ml de água. Adicione uma pitada de sal. Leve ao lume alto até ferver, depois reduza o lume para o mínimo, tape a panela e deixe cozinhar por cerca de 12-15 minutos, ou até que toda a água seja absorvida e o arroz esteja cozido e solto. Retire do lume e deixe repousar tapado.
2.  **Preparar a Carne com Tomate:** Enquanto o arroz cozinha, prepare a carne. Numa frigideira ou panela média, aqueça o azeite em lume médio-alto. Se estiver a usar, adicione a cebola picada e cozinhe até ficar translúcida (cerca de 3-4 minutos). Adicione o alho picado e cozinhe por mais 1 minuto, tendo cuidado para não queimar.
3.  Aumente ligeiramente o lume e adicione a carne. Doure a carne por todos os lados, mexendo ocasionalmente.
4.  Adicione o tomate picado em lata à panela. Tempere com sal e pimenta a gosto. Mexa bem para combinar.
5.  Reduza o lume para médio-baixo, tape a panela (deixe uma pequena abertura para o vapor sair) e deixe cozinhar lentamente por cerca de 10-15 minutos (se a carne for em tiras finas) ou 20-30 minutos (se for em cubos), ou até que a carne esteja macia e o molho tenha engrossado ligeiramente. Ajuste os temperos se necessário. Se o molho ficar muito seco, pode adicionar um pouco de água.
6.  **Servir:** Sirva o arroz cozido em pratos individuais. Coloque a carne com o molho de tomate por cima ou ao lado do arroz.

Bom apetite!

Esta é uma receita simples e reconfortante, e é possível adaptá-la para a Airfryer, embora o método de cozimento do molho com a carne seja um pouco diferente. A Airfryer é excelente para dourar e cozinhar a carne, mas não é ideal para cozinhar um molho líquido por um longo período como numa panela tradicional.

A adaptação envolverá cozinhar a carne e o molho em etapas separadas ou com uma abordagem diferente para obter o resultado desejado. O arroz continuará a ser cozido separadamente, pois a Airfryer não é adequada para cozinhar arroz da mesma forma que uma panela.

## Adaptação para Airfryer

Esta adaptação foca em usar a Airfryer para cozinhar a carne e, opcionalmente, pré-cozinhar os aromáticos (cebola e alho) e tomates antes de combiná-los para formar o molho. O molho será finalizado fora da Airfryer.

**Para 2 porções**

---

**Ingredientes:**

*   250g de carne de vaca (cortada em cubos pequenos ou tiras finas)
*   1 lata (aprox. 400g) de tomate picado em lata
*   200g de arroz (aproximadamente 1 chávena)
*   400ml de água (para cozinhar o arroz)
*   1 cebola pequena (opcional, picada)
*   1 dente de alho (opcional, picado)
*   1-2 colheres de sopa de azeite ou óleo vegetal (para a Airfryer)
*   Sal a gosto
*   Pimenta preta moída na hora a gosto
*   Uma pitada extra de azeite (para refogar cebola e alho se não o fizer na Airfryer)

---

**Equipamento:**

*   Airfryer
*   Panela pequena para cozinhar o arroz
*   Frigideira ou panela pequena para finalizar o molho

---

**Instruções:**

1.  **Preparar o Arroz:** Siga as instruções originais para cozinhar o arroz numa panela. Lave o arroz, adicione a água e sal, ferva, reduza o lume, tape e cozinhe até a água ser absorvida (cerca de 12-15 minutos). Deixe repousar tapado.

2.  **Preparar a Carne na Airfryer:**
    *   Pré-aqueça a Airfryer a 200°C por cerca de 5 minutos.
    *   Enquanto isso, tempere a carne com sal e pimenta. Se desejar, pode adicionar um pouco de alho em pó neste passo.
    *   Pulverize ou misture a carne com 1 colher de sopa de azeite para ajudar a dourar e evitar que grude.
    *   Coloque a carne na cesta da Airfryer numa única camada, se possível. Se necessário, cozinhe em lotes para não sobrepor a carne.
    *   Cozinhe a 200°C. O tempo dependerá do tamanho dos pedaços de carne e do ponto de cozedura desejado. Para cubos médios, pode levar de 6 a 10 minutos. Para tiras finas, pode ser um pouco menos, cerca de 7-8 minutos a 180°C inicialmente. Mexa a carne na metade do tempo para garantir um cozimento uniforme. A carne bovina geralmente leva cerca de 15 minutos a 200ºC na Airfryer. Para cubos grandes, pode levar até 25 minutos.

3.  **Preparar a Base do Molho (Método 1: Na Airfryer, opcional):**
    *   Num recipiente pequeno adequado para ir à Airfryer (como uma forma de bolo de silicone ou cerâmica), coloque a cebola picada e o alho picado com um fio de azeite.
    *   Leve à Airfryer a 180°C por cerca de 5-8 minutos, ou até a cebola ficar translúcida, mexendo na metade do tempo.
    *   Adicione o tomate picado a este recipiente e tempere com sal e pimenta. Pode ser necessário cozinhar por mais 5-10 minutos a 180°C para aquecer e apurar um pouco. Algumas receitas sugerem cozinhar tomate, cebola e alho na Airfryer a 200°C por 15-30 minutos para um molho mais assado e concentrado.

4.  **Preparar a Base do Molho (Método 2: Numa panela, mais tradicional para molhos):**
    *   Enquanto a carne cozinha, aqueça uma colher de sopa de azeite ou óleo numa frigideira ou panela pequena em lume médio.
    *   Se estiver a usar, adicione a cebola picada e cozinhe até ficar translúcida (cerca de 3-4 minutos).
    *   Adicione o alho picado e cozinhe por mais 1 minuto, tendo cuidado para não queimar.
    *   Adicione o tomate picado em lata à panela. Tempere com sal e pimenta a gosto. Mexa bem.
    *   Deixe cozinhar em lume brando por cerca de 5-10 minutos para apurar os sabores.

5.  **Combinar e Finalizar:**
    *   Retire a carne cozida da Airfryer e adicione-a à panela ou recipiente onde preparou a base do molho de tomate.
    *   Mexa delicadamente para envolver a carne no molho. Se o molho parecer muito espesso, pode adicionar uma ou duas colheres de sopa de água.
    *   Deixe aquecer por 1-2 minutos em lume baixo (se estiver numa panela) ou retorne rapidamente para a Airfryer num recipiente por 2-3 minutos a 180°C apenas para aquecer tudo junto (evite cozinhar por muito tempo na Airfryer com molho líquido).

6.  **Servir:** Sirva o arroz cozido em pratos individuais e coloque a carne com o molho de tomate por cima ou ao lado.

---

**Airfryer Adaptation**

*   **Temperatura Sugerida:** 200°C para dourar a carne inicialmente. Pode usar 180°C para cozinhar os aromáticos e tomates se optar por fazê-lo na Airfryer.
*   **Tempo Estimado:**
    *   Carne em cubos médios: 6-10 minutos a 200°C. (Pode variar dependendo do ponto desejado e tamanho dos cubos, até 25 minutos para cubos grandes).
    *   Carne em tiras finas: 7-8 minutos a 180°C inicialmente. Cerca de 10-15 minutos no total a 200°C, virando na metade.
    *   Cozinhar cebola e alho na Airfryer (opcional): 5-8 minutos a 180°C.
    *   Aquecer tomate com cebola/alho na Airfryer (opcional, para base do molho): 5-10 minutos a 180°C. Para um molho mais assado, 15-30 minutos a 200°C.
    *   Aquecer carne com molho na Airfryer (finalização): 2-3 minutos a 180°C.
*   **Instruções Específicas:**
    *   Pré-aqueça a Airfryer antes de adicionar a carne (cerca de 5 minutos a 200°C).
    *   Cozinhe a carne em uma única camada na cesta da Airfryer para garantir que doure uniformemente. Cozinhe em lotes, se necessário.
    *   Mexa a carne na metade do tempo de cozimento.
    *   Se optar por preparar a base do molho na Airfryer, use um recipiente adequado.
    *   O molho final com a carne é melhor ser finalizado rapidamente na Airfryer apenas para aquecer, ou numa panela no fogão, já que cozinhar molhos líquidos por muito tempo na Airfryer não é o ideal para a textura e concentração do molho.
    *   Tempere a carne antes de colocar na Airfryer. Adicionar sal à carne antes de selar pode fazer com que perca sucos, então considere adicionar o sal depois de a carne começar a dourar na Airfryer para maior suculência.
    *   Para dar sabor e evitar que a carne seque, pode-se adicionar um pouco de azeite. Para bife acebolado na Airfryer, adicionar manteiga pode ajudar na suculência.

Esta adaptação permite obter uma carne bem dourada e saborosa utilizando a Airfryer, enquanto o molho é preparado para garantir a consistência e profundidade de sabor desejadas.


Processing your request...
Personal Recipe Assistant: Your request details {'ingredients': ['fish', 'potatoes'], 'restrictions': [], 'meal_type': 'typical british meal', 'servings': 2, 'adaptations': ['Airfryer']}
Personal Recipe Assistant: Generating your recipe...
Personal Recipe Assistant: Processing Airfryer adaptation as requested...

Personal Recipe Assistant:


Okay, here is a recipe for a classic British meal using the ingredients you provided.

# Classic British Fish and Chips

A quintessential British takeaway dish, featuring flaky white fish encased in a crisp, golden batter, served alongside chunky, fluffy chips. Perfect for a comforting meal.

**Servings:** 2

**Ingredients:**

*   2 white fish fillets (such as cod, haddock, or pollock), about 150-200g each
*   2-3 large potatoes (about 400-500g), suitable for frying (like Maris Piper or King Edward)
*   1 cup plain flour, plus a little extra for dusting
*   1 teaspoon salt
*   ½ teaspoon black pepper
*   1 cup very cold water or beer (lager works well)
*   Vegetable oil or similar neutral oil, for deep frying (you'll need enough to submerge the fish and chips, likely 500ml to 1 litre depending on your pot size)
*   Optional: Malt vinegar, for serving

**Instructions:**

1.  **Prepare the Chips:** Peel the potatoes and cut them into chunky chips, about 1-1.5 cm thick. Rinse the cut chips thoroughly under cold water to remove excess starch. This helps them crisp up. Pat the chips completely dry with a clean towel or kitchen paper. This step is important to prevent oil splattering.
2.  **Par-fry the Chips:** Heat the oil in a large, deep frying pan or a deep pot (like a Dutch oven) over medium-high heat until it reaches about 130°C (265°F). Carefully add the chips in batches, making sure not to overcrowd the pan (frying too many at once lowers the oil temperature). Fry for about 5-8 minutes until they are softened but still pale and not browned. Remove the chips with a slotted spoon and drain them on a wire rack or plate lined with kitchen paper. Increase the oil temperature to about 180°C (350°F).
3.  **Make the Batter:** While the oil is heating for the second fry, prepare the batter. In a bowl, whisk together the 1 cup of plain flour, ½ teaspoon of salt, and ¼ teaspoon of black pepper. Gradually pour in the very cold water or beer, whisking constantly, until you have a smooth batter about the consistency of thick cream. Don't overmix; a few small lumps are fine.
4.  **Prepare the Fish:** Pat the fish fillets dry with kitchen paper. Lightly dust each fillet all over with a little extra plain flour. This helps the batter stick to the fish.
5.  **Fry the Fish:** Once the oil is at 180°C (350°F), carefully dip one fish fillet into the batter, ensuring it is fully coated. Lift it out, letting any excess batter drip off briefly, then gently lower it into the hot oil. Fry one fillet at a time if using a standard pan to maintain the oil temperature. Fry for 5-7 minutes, turning halfway through, until the batter is golden brown and crispy and the fish is cooked through and opaque (it should flake easily). Remove the fish with a slotted spoon and drain briefly on a wire rack or kitchen paper. Keep warm while you fry the second fillet and the chips.
6.  **Finish the Chips:** Return the par-fried chips to the hot oil (180°C / 350°F) in batches. Fry for another 2-4 minutes, or until they are golden brown and crispy on the outside and fluffy on the inside. Remove and drain on kitchen paper.
7.  **Serve:** Season the hot fish and chips generously with the remaining salt and pepper. Serve immediately. Traditionally, this dish is served with a sprinkle of malt vinegar.

Okay, here is the adaptation of your Classic British Fish and Chips recipe for cooking in an Airfryer.

Adapting a wet batter, like the one in the original recipe, for an air fryer can be tricky. The wet batter tends to drip through the air fryer basket before it has a chance to set, leading to a messy basket and a less crispy coating than traditional deep frying. For a better result with a crispy coating in the air fryer, a breadcrumb coating (often using panko for extra crispiness) is generally recommended.

This adaptation will guide you through making crispy chips and breaded fish using your air fryer.

## Airfryer Adaptation: Classic British Fish and Chips

This adaptation uses a breaded coating for the fish to achieve a crispy exterior in the air fryer, alongside air-fried chunky chips.

**Yields:** 2 servings
**Prep time:** 20 minutes
**Cook time:** Approx. 30-40 minutes (includes cooking chips and fish, likely in batches)

**Ingredients:**

*   2 white fish fillets (such as cod, haddock, or pollock), about 150-200g each
*   2-3 large potatoes (about 400-500g), suitable for chips
*   1-2 tablespoons vegetable oil or olive oil (for coating chips and fish)
*   Salt and black pepper to taste
*   ½ cup plain flour (for dusting/dredging)
*   1 large egg, beaten
*   1 cup Panko breadcrumbs (or regular breadcrumbs, Panko gives extra crispiness)
*   Optional: Malt vinegar, for serving

**Instructions:**

1.  **Prepare the Chips:** Peel the potatoes and cut them into chunky chips, about 1-1.5 cm thick. Rinse the cut chips thoroughly under cold water to remove excess starch. Pat the chips completely dry with a clean towel or kitchen paper. This is crucial for crispiness.
2.  **Season the Chips:** In a bowl, toss the dried chips with 1 tablespoon of oil and a generous pinch of salt and pepper. Ensure they are evenly coated.
3.  **Air Fry the Chips (First Batch):** Preheat your air fryer to 190°C (375°F). Place a single layer of seasoned chips in the air fryer basket. Do not overcrowd the basket; cook in batches if necessary. Air fry for 10-12 minutes.
4.  **Shake and Continue Frying Chips:** Open the air fryer basket and shake the chips well to redistribute them. Continue air frying at 190°C (375°F) for another 10-13 minutes, or until they are golden brown and crispy. You may need to adjust time based on your air fryer and chip thickness. Keep cooked chips warm (you can wrap them loosely in foil or place them in a very low oven). Repeat for any remaining batches of chips.
5.  **Prepare the Fish Breading Station:** Set up three shallow dishes. Put the ½ cup plain flour in the first dish and season with salt and pepper. Whisk the beaten egg in the second dish. Place the Panko breadcrumbs in the third dish, seasoning them with a little salt and pepper.
6.  **Prepare the Fish:** Pat the fish fillets very dry with kitchen paper.
7.  **Bread the Fish:** Take one fish fillet and dredge it first in the seasoned flour, shaking off any excess. Then dip it into the beaten egg, allowing excess to drip off. Finally, coat the fillet thoroughly in the seasoned Panko breadcrumbs, pressing gently to ensure the crumbs adhere. Repeat with the second fillet.
8.  **Air Fry the Fish:** Lightly brush or spray the breaded fish fillets with a little oil, or lightly spray the air fryer basket. Place the breaded fish fillets in the preheated air fryer basket in a single layer. Do not overcrowd; cook in batches if needed.
9.  **Cook the Fish:** Air fry the fish at 200°C (390°F) for 10-12 minutes, flipping gently halfway through the cooking time. The fish is cooked when the coating is golden brown and crispy, and the fish flakes easily with a fork (internal temperature should reach 63°C / 145°F). Cooking time may vary depending on the thickness of your fillets and air fryer model.
10. **Serve:** Season the hot fish and chips with extra salt and pepper to taste. Serve immediately, traditionally with malt vinegar.

**Airfryer Adaptation Summary:**

*   **Suggested Temperature:**
    *   Chips: 190°C (375°F)
    *   Fish: 200°C (390°F)
*   **Estimated Time:**
    *   Chips: 20-25 minutes total per batch
    *   Fish: 10-12 minutes total per batch
    *   Total time will depend on the number of batches required for both chips and fish.
*   **Specific Instructions:**
    *   **Chips:** Cut chips, rinse and *thoroughly* dry. Toss with oil and seasoning. Cook in a single layer, shaking the basket halfway through. Cook in batches.
    *   **Fish:** Replace the wet batter with a three-step breading process (flour, egg, seasoned breadcrumbs/panko). Pat fish very dry before breading. Lightly oil the breaded fish or the air fryer basket. Cook in a single layer, flipping halfway through. Cook in batches if necessary.
    *   **Preheating:** Preheat the air fryer for both chips and fish for best results.
    *   **Batch Cooking:** Essential for crispy results for both chips and fish. Do not overcrowd the basket.
    *   **Check for Doneness:** Times are estimates; check for desired crispiness and ensure fish is cooked through.


Recipe request (or type 'exit' to quit): exit

Enjoy your meal... See you soon!

 
